Benchmark Palantir runtime
---

In [ ]:
In this notebook, we benchmark the runtime of `run_palantir` function. We 

# Preliminaries

## Dependency notebooks

1. [MK_2020-10-16_gpcca.ipynb](MK_2020-10-16_gpcca.ipynb) - to extract the terminal states

## Import packages

In [9]:
# import standard packages
from pathlib import Path
from collections import defaultdict
from math import ceil
import sys
import pickle

import pandas as pd

# import single-cell packages
import scanpy as sc
import scvelo as scv
import palantir

# import utilities
import utils.utilities as ul

## Print package versions for reproducibility

In [6]:
sc.logging.print_header()
palantir.__version__

scanpy==1.6.0 anndata==0.7.4 umap==0.4.6 numpy==1.19.2 scipy==1.5.2 pandas==1.1.3 scikit-learn==0.23.2 statsmodels==0.12.0 python-igraph==0.8.2 louvain==0.7.0 leidenalg==0.8.2


'1.0.0'

## Set up paths

In [8]:
sys.path.insert(0, "../../..")  # this depends on the notebook depth and must be adapted per notebook

from paths import DATA_DIR

## Load the data

Load the raw data.

In [2]:
adata = sc.read(DATA_DIR / "morris_data" / "adata.h5ad")
del adata.layers  # we don't need these
adata

AnnData object with n_obs × n_vars = 104679 × 22630
    obs: 'batch'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand'

### Load supplementary information data

This is to get the `Reprogramming Day` annotation, which we use to get the root cell.

In [3]:
annot = pd.read_csv(DATA_DIR  "morris_data" / "annotations" / "supp_table_4.csv", index_col=0, header=2)
annot.head()

,Genes,UMIs,% Mitochondrial RNA,Cell Cycle Phase,Timecourse,Reprogramming Day,CellTagMEF,CellTagD3,CellTagD13,CellTagMEF.1,CellTagD3.1,CellTagD13.1
Cell ID,,,,,,,,,,,,
_HF-1_AGAATAGGTAGCGCTC-1,"4,551","6,497",7.30,G1,1,0,16.0,NaN,NaN,16.0,NaN,NaN
_HF-1_AGGTCCGTCAACACGT-1,"4,625","6,726",7.39,G1,1,0,20.0,NaN,NaN,20.0,NaN,NaN
_HF-1_CACATAGGTTCCCTTG-1,"5,488","6,869",6.03,G1,1,0,31.0,NaN,NaN,31.0,NaN,NaN
_HF-1_CCAGCGACAGAGCCAA-1,"5,549","6,995",7.07,G1,1,0,39.0,NaN,NaN,39.0,NaN,NaN
_HF-1_CCGGTAGCACGTGAGA-1,"4,901","6,541",7.03,G1,1,0,NaN,NaN,NaN,NaN,NaN,NaN


### Load the subsets and splits

In [ ]:
dfs = ul.get_split(DATA_DIR / "morris_data" / "splits")
list(dfs.keys())

## Define utility functions

In [4]:
def clean_orig_names(series):
    return np.array(list(map(np.array, series.str.split(":"))))[:, 1]

In [5]:
def add_annotations(adata, annot):
    adata.obs['genes_cleaned'] = clean_orig_names(adata.obs.index)
    assert len(set(adata.obs['genes_cleaned'])) == len(adata)
    
    annot['genes_cleaned'] = np.array(list(map(np.array, annot.index.str.split("_"))))[:, 2]
    annot['genes_cleaned'] = annot['genes_cleaned'].str.replace("-", "x-")
    
    tmp = adata.obs.merge(annot, how='left', on='genes_cleaned')
    tmp.drop_duplicates('genes_cleaned', inplace=True)
    tmp.set_index('genes_cleaned', drop=True, inplace=True)

    adata.obs = tmp
    adata.obs['Reprogramming Day'] = adata.obs['Reprogramming Day'].astype('category')
    
    
def select_root_cell(adata):
    obs = adata.obs['Reprogramming Day']
    min_val = np.nanmin(obs.cat.categories)
    
    return obs[obs == min_val].index[0]

In [6]:
def load_cellrank_final_states(adata, data, size, col):
    try:
        index = clean_orig_names(data[size][col]['main_states'].index)
        valid_ixs = np.isin(index, adata.obs.index)

        x = data[size][col]['lin_probs'][valid_ixs, :]
        x = pd.DataFrame(x, index=index[valid_ixs])

        if len(index) < 3:
            return None

        ixs = []
        for lin in range(x.shape[1]):  # be extra careful
            y = x[~np.isin(x.index, ixs)]

            assert len(y) + len(ixs) == x.shape[0], "Sanity check failed"

            # we select the most likely cell from each terminal state
            ix = np.argmax(y.values[:, lin])
            ixs.append(y.index[ix])

        return ixs
    except Exception as e:
        print(f"Unexpected error: `{e}`.")
        return None

In [10]:
def palantir_preprocess(adata):
    sc.pp.filter_genes(adata, min_cells=10)
    sc.pp.normalize_total(adata)
    sc.pp.log1p(adata)
    sc.pp.highly_variable_genes(adata, flavor='cell_ranger', n_top_genes=1500)
    
    print("Running PCA")
    n_comps = 300
    sc.pp.pca(adata, use_highly_variable=True, n_comps=n_comps)

    print("Diff maps")
    dm_res = palantir.utils.run_diffusion_maps(pd.DataFrame(adata.obsm['X_pca'][:, :n_comps],
                                                            index=adata.obs_names))
    
    print("MS space")
    ms_data = palantir.utils.determine_multiscale_space(dm_res)
    
    return ms_data

In [11]:
def benchmark_palantir(adata, dfs, annot, final_states_path):
    res = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))
    run_palantir = ul.timeit(palantir.core.run_palantir)
    
    with open(final_states_path, 'rb') as fin:
        # load the terminal states from cellrank
        fs_data = pickle.load(fin)
    
    for size, split in dfs.items():
        for col in split.columns:
            try:
                print(f"Subsetting data to `{size}`, split `{col}`.")
                ixs = split[col].values
                bdata = adata[ixs].copy()
                add_annotations(bdata, annot)
                
                assert bdata.n_obs == size
                
                root_cell = select_root_cell(bdata)
                final_states = load_cellrank_final_states(bdata, fs_data, size, col)
                                
                print("Preprocessing")
                ms_data = palantir_preprocess(bdata)
                
                
                print(f"Running with no terminal states, root_cell=`{root_cell}`")
                _, time_no_ts = run_palantir(ms_data,
                                             root_cell,
                                             terminal_states=None,
                                             knn=30,
                                             num_waypoints=int(ceil(size * 0.15)),
                                             n_jobs=-1,
                                             scale_components=False,
                                             use_early_cell_as_start=True)
                
                res[size][col]['core_no_ts'] = time_no_ts
                
                if final_states is None:
                    print("No final states found, skipping")
                    continue
                elif root_cell in final_states:
                    print("Root cell is in final states, skipping")
                    continue
                                  
                # this is the data we're using the figures
                print(f"Running with CellRank terminal states `root_cell={root_cell}` and "
                      f"`final_states={final_states}`")
                _, time_ts = run_palantir(ms_data,
                                          root_cell,
                                          terminal_states=final_states,
                                          knn=30,
                                          num_waypoints=int(ceil(size * 0.15)),
                                          n_jobs=-1,
                                          scale_components=False,
                                          use_early_cell_as_start=True)
                
                res[size][col]['core_ts'] = time_ts
                ul.save_results(res, DATA_DIR / "runtime_analysis" / "palantir.pickle")
            except Exception as e:
                print(f"Unable to run `Palantir` with size `{size}` on split `{col}`. Reason: `{e}`.")
                continue
                
    return res

# Run the benchmarks

In [ ]:
res_pala = benchmark_palantir(adata, dfs, annot,
                              final_states_path=DATA_DIR / "runtime_analysis" / "gpcca.pickle")

Subsetting data to `70000`, split `0`.
Preprocessing
Running PCA
Diff maps
Determing nearest neighbor graph...
MS space
Running with no terminal states, root_cell=`GTCCTCATCCTGTAGAx-1`
Sampling and flocking waypoints...
Time for determining waypoints: 9.7591912428538 minutes
Determining pseudotime...
Shortest path distances using 30-nearest neighbor graph...
Time for shortest paths: 16.08590335448583 minutes
Iteratively refining the pseudotime...
Correlation at iteration 1: 1.0000
Entropy and branch probabilities...
Markov chain construction...
Identification of terminal states...
Computing fundamental matrix and absorption probabilities...
Project results to all cells...
Running with CellRank terminal states `root_cell=GTCCTCATCCTGTAGAx-1` and `final_states=['GGTGTTAGTCCATGATx-13', 'CGAGAAGGTGTGTGCCx-8', 'CTCACACTCCACGACGx-5']`
Sampling and flocking waypoints...
Time for determining waypoints: 9.190261618296306 minutes
Determining pseudotime...
Shortest path distances using 30-nearest

## Save the results

In [ ]:
ul.save_results(res_pala, DATA_DIR / "runtime_analysis" / "palantir.pickle")